In [1]:
import os
import sys
sys.path.append(os.path.abspath('') + '/..')

# imports

In [2]:
import pandas as pd 
import joblib

from data import ImplicitData, getBucketsHoldouts
from eval_implicit import EvaluateHoldouts, EvalPrequential 
from recommenders_implicit import ISGD

from plot_utils import recall_heatmap
from data_utils.transfer_learning_scores import *
from dataset_evaluation_utils import * 


import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')

# import plotly.offline as py
# pd.options.plotting.backend = "plotly"
# py.init_notebook_mode() # graphs charts inline (IPython).

a4_dims = (11.7, 8.27)

# functions

In [3]:
dataset_folderpath = '../datasets/lastfm1b/'

dataset_name = 'LastFM1b'

dump_foldername ='run_online/lastfm1b_dump/'
_, base_outputpath, _, _, _ = get_folderpaths(dump_foldername)


period=['2011-07', '2014-01']
sample = 'sample_'+str(period[0])+'_until_'+str(period[1])

sample_version_dump_foldername=dump_foldername+sample+'/'
filename = 'tracks_inter_merged_coldstart_11M'


# rule: what/which_data_set/sample_version/what/
images_path, output_path, heatmaps_path, diversity_graphpath, diversity_filepath = get_folderpaths(sample_version_dump_foldername)


In [4]:
sample_year_month = [(period[0], '%Y-%m'), (period[1], '%Y-%m')]
interval_type = 'S'

use_data_unique_users = False
frequent_users_thr = 0.75 
cold_start_buckets = 1
to_grid_search = False
num_factors = 100
num_iter = 6
learn_rate = 0.1
regularization = 0.1
random_seed = 10


user_col = 'user_id'
item_col = 'item_id'

In [ ]:
# buckets = joblib.load(output_path+filename+'_semesterly_buckets.joblib')
# buckets

In [5]:
# holdouts = joblib.load(output_path+filename+'_semesterly_holdouts.joblib')
# holdouts

In [6]:
frequent_users = joblib.load(output_path+filename+'_frequent_users_'+str(frequent_users_thr)+'.joblib')
frequent_users[2][:5]

['1000557', '14975516', '14972179', '4931051', '14912953']

In [9]:
frequent_users

[None,
 None,
 ['1000557',
  '14975516',
  '14972179',
  '4931051',
  '14912953',
  '14858093',
  '4941484',
  '14827445',
  '14718892',
  '4949750',
  '14678368',
  '4957456',
  '14600181',
  '4963478',
  '14561717',
  '4965724',
  '14534500',
  '14480015',
  '14446160',
  '14439534',
  '14370885',
  '5039141',
  '5060934',
  '5072629',
  '14577216',
  '15012946',
  '4891787',
  '16399131',
  '16381211',
  '16322178',
  '16314126',
  '16310839',
  '16288734',
  '4812967',
  '16103870',
  '16087362',
  '16081844',
  '16056236',
  '4830083',
  '15922728',
  '15047644',
  '15791642',
  '15788756',
  '15765121',
  '4845079',
  '15622120',
  '15585888',
  '4856498',
  '15503736',
  '15496640',
  '15441836',
  '15397460',
  '15306304',
  '15813922',
  '5113926',
  '14237425',
  '5444349',
  '5454192',
  '5457852',
  '13807133',
  '13787525',
  '5464534',
  '13780570',
  '5472324',
  '5481913',
  '13778677',
  '5484708',
  '13776857',
  '5489275',
  '13753711',
  '5492034',
  '5544243',
  '5

In [7]:
interactions_df = pd.read_csv(output_path+filename+'_interactions_df.csv', index_col=0)
interactions_df['date'] = interactions_df['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
interactions_df.columns = ['user_id','item_id','timestamp','date']
interactions_df.head()

,user_id,item_id,timestamp,date
0,31435741,4,1385212958,2013-11-01
1,31435741,17,1381867823,2013-10-01
2,31435741,19,1381867431,2013-10-01
3,31435741,53,1370977938,2013-06-01
4,31435741,86,1369853851,2013-05-01


In [9]:
interactions_df['user_id'].dtype

dtype('int64')

In [10]:
frequent_users[2] = list(map(int, frequent_users[2]))
frequent_users[2][:5]

[1000557, 14975516, 14972179, 4931051, 14912953]

In [11]:
frequent_users_idx = 2
interval_type ='QS'
intervals_path = output_path+filename+'_semesters.joblib'
intervals = joblib.load(intervals_path)


print('\nSTAGE -> getBucketsHoldouts')
buckets, holdouts = getBucketsHoldouts( data = interactions_df,
                                        user_col = user_col,
                                        item_col = item_col,
                                        frequent_users = frequent_users[frequent_users_idx],
                                        interval_type = interval_type,
                                        intervals = intervals, 
                                        cold_start_buckets = cold_start_buckets)


STAGE -> getBucketsHoldouts
Creating buckets. . .
Creating holdouts. . .
cold start bucket
idx.sum() 1014,0+ inter user,8267909,idx.sum() 453,0+ inter user,1814276,idx.sum() 23,0+ inter user,3533644,idx.sum() 139,0+ inter user,7828996,idx.sum() 583,0+ inter user,3267139,idx.sum() 439,0+ inter user,6988241,idx.sum() 768,0+ inter user,3456071,idx.sum() 303,0+ inter user,6575920,idx.sum() 1449,0+ inter user,3205243,idx.sum() 336,0+ inter user,541991,idx.sum() 120,0+ inter user,10741197,idx.sum() 928,0+ inter user,7204877,idx.sum() 811,0+ inter user,3642829,idx.sum() 797,0+ inter user,3252664,idx.sum() 2035,0+ inter user,2981702,idx.sum() 403,0+ inter user,5247254,idx.sum() 313,0+ inter user,6209646,idx.sum() 1466,0+ inter user,7301050,idx.sum() 456,0+ inter user,580088,idx.sum() 382,0+ inter user,9262608,idx.sum() 608,0+ inter user,4878557,idx.sum() 0,idx.sum() 1232,0+ inter user,8320770,idx.sum() 213,0+ inter user,2896126,idx.sum() 462,0+ inter user,7055137,idx.sum() 451,0+ inter user,2

KeyboardInterrupt: 